In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd

In [3]:
!pip3 install vncorenlp
import numpy as np
# %env DATA_DIR=./data/squad 
# !wget -P $DATA_DIR https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v1.1.json
# !wget -P $DATA_DIR https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v1.1.json
!pip install transformers==3.5.1
!pip install torch==1.4.0
# !pip install sentencepiece

In [4]:
!git clone https://github.com/vncorenlp/VnCoreNLP.git

fatal: destination path 'VnCoreNLP' already exists and is not an empty directory.


In [5]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow_datasets.question_answering import xquad
import json
import numpy as np

In [6]:
from vncorenlp import VnCoreNLP
annotator = VnCoreNLP("/content/VnCoreNLP/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m') 

In [ ]:
#df = tfds.as_dataframe(ds, dataset.info)

In [35]:
customer_json_file = '/content/drive/MyDrive/Colab Notebooks/dev_ViQuAD.json'
df = pd.read_json(customer_json_file,
   convert_dates=True)

In [36]:
import json
  
# Opening JSON file
with open(customer_json_file) as json_file:
    data = json.load(json_file)
  
    # Print the type of data variable
    print("Type:", type(data))


Type: <class 'dict'>


In [80]:
# data['data'][0]['paragraphs'][0]['context']
# data['data'][0]['paragraphs'][0]['qas'][0]['answers'][0]['answer_start']
# data['data'][0]['paragraphs'][0]['qas'][0]['answers'][0]['text']
# data['data'][0]['paragraphs'][0]['qas'][0]['id']
# data['data'][0]['paragraphs'][0]['qas'][0]['question']
# data['data'][0]['title']

dic = {'answer/answer_start': '', 'answers/text': '', 'context': '', 'id': '', 'question': '', 'title': ''}
dic['answer/answer_start'] = [data['data'][0]['paragraphs'][0]['qas'][0]['answers'][0]['answer_start']]
dic['answers/text'] = data['data'][0]['paragraphs'][0]['qas'][0]['answers'][0]['text']
dic['context'] = data['data'][0]['paragraphs'][0]['context']
dic['id'] = data['data'][0]['paragraphs'][0]['qas'][0]['id']
dic['question'] = data['data'][0]['paragraphs'][0]['qas'][0]['question']
dic['title'] = data['data'][0]['title']
dic

# #print(data['data'][0]['paragraphs'][0]['qas'][0]['answers'][0].keys())
# data['data'][0]['paragraphs'][0]

{'answer/answer_start': [507],
 'answers/text': 'Lâm Bá Kiệt',
 'context': 'Phạm Văn Đồng (1 tháng 3 năm 1906 – 29 tháng 4 năm 2000) là Thủ tướng đầu tiên của nước Cộng hòa Xã hội chủ nghĩa Việt Nam từ năm 1976 (từ năm 1981 gọi là Chủ tịch Hội đồng Bộ trưởng) cho đến khi nghỉ hưu năm 1987. Trước đó ông từng giữ chức vụ Thủ tướng Chính phủ Việt Nam Dân chủ Cộng hòa từ năm 1955 đến năm 1976. Ông là vị Thủ tướng Việt Nam tại vị lâu nhất (1955–1987). Ông là học trò, cộng sự của Chủ tịch Hồ Chí Minh. Ông có tên gọi thân mật là Tô, đây từng là bí danh của ông. Ông còn có tên gọi là Lâm Bá Kiệt khi làm Phó chủ nhiệm cơ quan Biện sự xứ tại Quế Lâm (Chủ nhiệm là Hồ Học Lãm).',
 'id': 'uit_01__05272_0_1',
 'question': 'Tên gọi nào được Phạm Văn Đồng sử dụng khi làm Phó chủ nhiệm cơ quan Biện sự xứ tại Quế Lâm?',
 'title': 'Phạm Văn Đồng'}

In [37]:
ls = []
# dic = {'answers/answer_start': '', 'answers/text': '', 'context': '', 'id': '', 'question': '', 'title': ''}

for title_group in data['data']:  # 'title_group' keys: 'title', 'paragraphs'
  for context_group in title_group['paragraphs']:  # 'context_group' keys: 'qas', 'context'
    for question_group in context_group['qas']:  # 'question_group' keys: 'question', 'answers', 'id'
      for text_group in question_group['answers']:  # 'text_group' keys: 'answer_start', 'text'
        dic = {}
        dic['answers/answer_start'] = [text_group['answer_start']]
        dic['answers/text'] = text_group['text']
        dic['context'] = context_group['context']
        dic['id'] = question_group['id']
        dic['question'] = question_group['question']
        dic['title'] = title_group['title']

        ls.append(dic)




In [38]:
df = pd.DataFrame(ls)
df.head(20)

,answers/answer_start,answers/text,context,id,question,title
0,[6],nằm ở điểm gặp nhau của các hành trình thương ...,Paris nằm ở điểm gặp nhau của các hành trình t...,uit_01__00256_0_1,Vị trí địa lý của Paris có gì đặc biệt?,Paris
1,[12],điểm gặp nhau của các hành trình thương mại đư...,Paris nằm ở điểm gặp nhau của các hành trình t...,uit_01__00256_0_1,Vị trí địa lý của Paris có gì đặc biệt?,Paris
2,[12],điểm gặp nhau của các hành trình thương mại đư...,Paris nằm ở điểm gặp nhau của các hành trình t...,uit_01__00256_0_1,Vị trí địa lý của Paris có gì đặc biệt?,Paris
3,[12],điểm gặp nhau của các hành trình thương mại đư...,Paris nằm ở điểm gặp nhau của các hành trình t...,uit_01__00256_0_1,Vị trí địa lý của Paris có gì đặc biệt?,Paris
4,[109],nông nghiệp,Paris nằm ở điểm gặp nhau của các hành trình t...,uit_01__00256_0_2,Kinh tế xung quanh kinh đô ánh sáng mạnh về gì?,Paris
5,[323],giáo dục và nghệ thuật,Paris nằm ở điểm gặp nhau của các hành trình t...,uit_01__00256_0_2,Kinh tế xung quanh kinh đô ánh sáng mạnh về gì?,Paris
6,[323],giáo dục và nghệ thuật,Paris nằm ở điểm gặp nhau của các hành trình t...,uit_01__00256_0_2,Kinh tế xung quanh kinh đô ánh sáng mạnh về gì?,Paris
7,[323],giáo dục và nghệ thuật,Paris nằm ở điểm gặp nhau của các hành trình t...,uit_01__00256_0_2,Kinh tế xung quanh kinh đô ánh sáng mạnh về gì?,Paris
8,[358],Paris,Paris nằm ở điểm gặp nhau của các hành trình t...,uit_01__00256_0_3,Cuộc cách mạng Pháp diễn ra ở đâu?,Paris
9,[358],Paris,Paris nằm ở điểm gặp nhau của các hành trình t...,uit_01__00256_0_3,Cuộc cách mạng Pháp diễn ra ở đâu?,Paris


In [39]:
#For training sample
df = df.applymap(lambda x: x.decode('utf-8', 'ignore') if isinstance(x, bytes) else x)
df = df.applymap(lambda x: x[0].decode('utf-8', 'ignore') if isinstance(x, np.ndarray) and isinstance(x[0], bytes) else x)

In [ ]:
# # For dev/test sample
# df = df.applymap(lambda x: x.decode('utf-8', 'ignore') if isinstance(x, bytes) else x)
# df = df.applymap(lambda x: [x_i.decode('utf-8', 'ignore') for x_i in x] if isinstance(x, np.ndarray) and isinstance(x[0], bytes) else x)

In [40]:
df.head(5)

,answers/answer_start,answers/text,context,id,question,title
0,[6],nằm ở điểm gặp nhau của các hành trình thương ...,Paris nằm ở điểm gặp nhau của các hành trình t...,uit_01__00256_0_1,Vị trí địa lý của Paris có gì đặc biệt?,Paris
1,[12],điểm gặp nhau của các hành trình thương mại đư...,Paris nằm ở điểm gặp nhau của các hành trình t...,uit_01__00256_0_1,Vị trí địa lý của Paris có gì đặc biệt?,Paris
2,[12],điểm gặp nhau của các hành trình thương mại đư...,Paris nằm ở điểm gặp nhau của các hành trình t...,uit_01__00256_0_1,Vị trí địa lý của Paris có gì đặc biệt?,Paris
3,[12],điểm gặp nhau của các hành trình thương mại đư...,Paris nằm ở điểm gặp nhau của các hành trình t...,uit_01__00256_0_1,Vị trí địa lý của Paris có gì đặc biệt?,Paris
4,[109],nông nghiệp,Paris nằm ở điểm gặp nhau của các hành trình t...,uit_01__00256_0_2,Kinh tế xung quanh kinh đô ánh sáng mạnh về gì?,Paris


In [41]:
def check_loop_string(symbol, string_a, len_s):
  if len(string_a) == 0:
    return len_s
  if symbol == string_a[0]:
    return check_loop_string(symbol, string_a[1:], len_s+1)
  return len_s

In [42]:
special_symbol = ['"',"'", '[',']','(',')','/','\\','{','}','&', '*', '?', ';', ':', '.', ',', '%', '!', '#']
def refine_senc(ori_sen,list_sens):
  senc = [word for sen in list_sens for word in sen]
  for idx, word in enumerate(senc):
    if word[0] == "_" and len(word) > 1:
      senc[idx] = word[1:]
    for symbol in special_symbol:
      if senc[idx].find(symbol) != 0 and senc[idx].find(symbol) != -1 or (senc[idx].find(symbol) == 0 and len(senc[idx])!=0) or (senc[idx].find("_") == 1 and len(senc[idx])==3):
        senc[idx] = senc[idx].replace("_", "")
  senc = "".join(senc)
  for idx, char in enumerate(ori_sen):
    if (char == " " and senc[idx] != "_") or (char == " " and (check_loop_string("_", senc[idx:], 0)>1)):
      senc = senc[:idx] + ' ' + senc[idx:]
  return senc

In [43]:
import transformers
from transformers.data.processors.squad import SquadV1Processor
processor = SquadV1Processor()

In [44]:
structure_squadv1 = {
    'version' : '1.1',
    'data' : []
}

In [45]:
from tqdm import tqdm
import pandas as pd

In [46]:
for idx, row in tqdm(df.iterrows()):
  # answer_text = [refine_senc(row['answers/text'][i], annotator.tokenize(row['answers/text'][i])) for i in range(len(row['answers/answer_start']))]
  answer_text = [refine_senc(row['answers/text'], annotator.tokenize(row['answers/text']))]
  context = refine_senc(row['context'], annotator.tokenize(row['context']))
  sample = {
    'title': row['title'],
    'paragraphs': [
    {
        'context': context,
        'qas': [{ 'id': row['id'],
                  'question': refine_senc(row['question'], annotator.tokenize(row['question'])),
                  'answers': [{ 'answer_start': int(row['answers/answer_start'][i]),
                                'text': context[row['answers/answer_start'][i]:row['answers/answer_start'][i]+len(answer_text[i])] } for i in range(len(row['answers/answer_start']))] 
                }]
    }]}
  structure_squadv1['data'].append(sample)

9140it [02:43, 56.04it/s]


In [47]:
  structure_squadv1['data'][1000]

{'paragraphs': [{'context': "Phần_lớn những tập_đoàn truyền_thông của Pháp đều nằm tại Paris và thành_phố cũng có một_số tờ báo riêng. Le_Parisien là nhật_báo chính của Paris. 20 phút và Métro là hai nhật_báo miễn_phí, không có nguồn_gốc ở Paris, nhưng được nhiều người_dân thành_phố đọc. Những hành_khách đọc các tờ báo miền phí trên phương_tiện giao_thông công_cộng trở_thành một hình_ảnh hàng ngày của thành_phố. Về văn_hoá, có tờ L'Officiel des spectacles đăng các chương_trình, và tờ Paris frimousse phát_hành hàng tháng về các hoạt_động văn_hoá cho thiếu_nhi. Trước_đây còn hai tờ báo khác cũng đăng tin tương_tự là Zurban và Le_Pariscope.",
   'qas': [{'answers': [{'answer_start': 593,
       'text': 'Zurban và Le_Pariscope'}],
     'id': 'uit_01__00256_87_3',
     'question': 'Hai hãng nhật_báo văn_hoá ở Paris đã ngưng hoạt_động tên gì?'}]}],
 'title': 'Paris'}

In [48]:
df.iloc[1000]

answers/answer_start                                                [593]
answers/text                                       Zurban và Le Pariscope
context                 Phần lớn những tập đoàn truyền thông của Pháp ...
id                                                     uit_01__00256_87_3
question                Hai hãng nhật báo văn hóa ở Paris đã ngưng hoạ...
title                                                               Paris
Name: 1000, dtype: object

In [49]:
with open('/content/drive/MyDrive/Colab Notebooks/dev_ViQuAD_ws.json', 'w') as outfile:
    json.dump(structure_squadv1, outfile)